In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from scipy.spatial.distance import cdist, euclidean
from scipy.stats import truncnorm
from scipy.sparse import csr_matrix
from time import time
import networkx as nx

from first_second_stages import *



,Longitude,Latitude
0,24.651095,58.353754
1,16.918554,49.446910
2,31.950043,47.198117
3,265.853989,57.609891
4,112.258028,49.075618
...,...,...
1255,272.320254,24.123792
1256,88.678381,36.099085
1257,20.173602,69.208967
1258,240.575109,80.531734
